* Data intensive application is typically built from standard building block
    - Store data so that we can find it later (database)
    - Store expensive operation to speed up reads (cache)
    - Allow user to search by keyword or filter in various ways (search indexes)
    - Send a message to another process, to handle asynchronously (stream processing)
    - Periodically crunch large amount of accumulated data (Batch processing)

![](images/data_arch.JPG)

## Reliability
* System should continue to work properly (Performing correct function with proper performance) even in the face of adversity.
* Application should perform function that the user expected
* It can tolerate the user making mistakes or using software unexpected ways
* Its performance is good enough for the required user case, under the expected load and volume
* Prevent unauthorized access and abuse.
* To check and test system reliability, we can intentionally stop some component. Netflix Chaos Monkey
* Hardware failure
    * Redundancy, Dual power supply, hot-swapped CPU
* Software error: like Linux kernel leap second. Some process eat up shared resources, service that system depends on slows down. Such happen suddenly due to assumptions of software and suddenly that assumption become false. Careful assumption and interaction design. Testing, measuring, monitoring.

## Scalability
* As the system grows, there should be reasonable ways of dealing with growth.
* If system grows in certain way, what are our options to coping with growth.
* How can we add computing resources to handle additional loads.
* Load can be defined as requests per seconds to web server, ration of read and write in db, number of simultanosly active user in chat room, hit rate in cache
* Example: Twitter
    - Post tweet: a user publish a tweet to their followers
    - Home timeline: A user can view tweets posted by the people they follow
* Posting a tweet will just add new tweet in global collection of tweets. When user request home timeline, lookup all the people they follow, find all the tweets for each of those users and merge them sorted by time

```
SELECT tweet.*, users.* FROM tweets
JOIN users ON tweets.sender_id = user.id
JOIN follows ON follows.fllowee_id = users.id
WHERE follows.follower_id = current_user
```
![](images/twitter_1.png)
* Other way is maintain cache of each user's timeline, when user post a tweet look up all the people who follow him and insert the new tweet into each of their home timeline cache. Reading of home timeline is now cheap.

![](images/twitter_2.png)

* Now twitter is using hybrid approach, most of the time second approach works, but for users whoes followers are (Barak obama) very large, their tweets are fetched separately in merged with other tweets.

* What happen when resources are same and load is increased?
* How much we have to increase resources to keep up performance?
* Performance measure on response time, or throughput.
* Resource time in what the client sees, process the request + network delay + queueing delays.
* Latency is duration that a request is waiting to be handled.
* Using mean of all response time, is not good measure. We should use percentile. And p50 shows that 50% of users experienced lower response time than that, Extreme late in response can be seen at P95 and more.
* SLA (service level agreements) also define as median response time is less than 200 ms and 99th percentile under 1 s.
* For single server system, one lengthly request can slow down all later queued request.


## Maintainability
* Many people maintain and work with application to add and support features, they should be able to work productively.



------------

# Data Models and Query Language



* How data is viewed and represented?
    - As application developer, we look at real world and model it in terms of data structure, objects, API that manipulate those objects and data structures.
    - To store data structure we can use JSON, XML, tables in relational database
    - Above can represent in bytes in memory, on disk or on network.
    - On other lower layer hardware engineer figure out how to represent byte in electrical currents etc.
* Top layer hide the abstraction of lower layer.

### Relational vs Document model
* In relational model data is organized in relation(table) which is unordered collection of rows(tuples).
* Transaction processing : entering sales or banking transaction, airline reservation, stock-keeping in warehouse)
* Batch processing: Customer invoicing, payroll, reporting
* We uses object oriented model in programming, but data is in relational model. We have Object relational mapping ORM, like activerecord and hibernate to overcome this problem.
* Resume
    - User: user_id, first_name, last_name
    - Job: multiple job, education: multiple education at different time, contact_info :multiple numbers
    - This is one to many relationship. In relational model we can normalize it and user, education, contact and job in different table and use foreign key to reference user table
    - Some SQL implementation allow multi-values data to be stored within  single row
    - Encode job, education , contact as JSON or XML store it on text column in database and let application interpret structure and content. We can not use database to query such encoded columns.
    ![](images/resume.PNG)

* Use of fixed values and id in region table and industry type
    - Consistent style and spelling across profile
    - Avoid ambiguity
    - Ease of updating as stored at one place
    - Localization support - When site translated into other language
    - Better search
* Using this way we can normalize the data. Which leads to many to one relationship.

* Store as JSON document, supported by document oriented database like MongoDB, RethinkDB, CouchDB, Espresso
![](images/json_resume.PNG)

* JSON offers locality than multi table schema.TO fetch profile in relational we have to fire multiple queries or perform large JOIN. In JSON everything is at one place.
* In JSON model, we might not have support to JOIN, to fetch the region name bu id, application code's responsibility to fire separate query of keep mapping in cache memory.

* We can also add organization or school table and add id to resume instead simple text.
* Recommendation, has to have just user id not all info of user who wrote it.

* Document model has schema flexibility, better performance due to locality, for some application it is closer to data structure used by an application.
* Relational model provide better support for join, many to one and many to many relationship.
* If data has tree like structure, go with document model. In document model we can not refer directly to a nested item within document. We must have to go like second item in the list of jobs for user 251.
* If application is using many to many relationship, document model is less effective as we might denormalize data but now we have to manage consistency. Other way to join by application, but it create another responsibility for application code. ANd such join is slower. For highly inter connected data relational model is acceptable.
* XML does support optional schema validation.
* No schema means any key, value can be added to document and can missing in document. Here schema is on read, schema is implicit, Structure only interpreted when data is read. In relational database schema on write, database ensure all data conforms to it.
* Update in schema oriented and schema less,
![](images/json_update.PNG)
![](images/SQL_update.PNG)

* When application needs entire data of resume data locality is useful. Some implementation allow data locality in relational database by interleaving table's row with parent table

### Query languages for data

* SQL is declarative and other IMS, CODASYL are imperative. In imperative we loop over the data and filter out data which we need

* In SQL we only define

```
SELECT * FROM animals WHERE family = 'Sharks';
```
* Imperative code tells computer step by step instructions. In declarative we define pattern of data we need, but not how to achieve that goal. Database system's optimizer decide which indexes and which join method to use.
* And because of it database can optimize code.
* Declarative code can be parallelize easily. Imperative is hard to parallelize as it must be executed in order.

### Graph like data model
* 1 to many relationship is good with document model. Many to many can be modeled using graph.
* Vertices (nodes, entities) and edges(relationship or arcs).
* Example, social graph, web graph, road and rail neteork.
* Facebook maintains a single graph with many different types of vertices and edges: vertices represent people, locations, events, checkins, and comments made by users; edges indicate which people are friends with each other, which checkin happened in which location, who commented on which post, who attended which event, and so on.

#### Property Graph model
* Neo4j, Titan,infiniteGraph
* Each vertex consists of unique identifier, set of outgoing and incoming edges, collection of properties like key value pairs
* Each edge has unique identifier, vertex at which edge start and ends (tail and head vertex respectively). Label to describe kind of relationship between 2 vertex., collection of properties (key-val pair)
* Implementation using relational model
![](images/graph_model.PNG)

#### Triple-store model
* Datomic, AllegroGraph


## Storage and Retrieval

* We need an index to find the value for particular key in the database. Index helps to locate data we want. To search data in different way, we need different index. Keeping index has overhead on write. 

#### Hash index
* Index for key-value data. Every key is mapped to byte offset in data file. It is in-memory indexing. Ex. Bitcask (defualt storage engine for Riak) uses it. Condition is all key must be accomodated in RAM. Useful when not too much key, but too much write for values. Ex. URL access counts.

# Replication

* Keeping copy of data on multiple machines that are connected via network .
* Useful
    - To keep data geographically closer
    - To allow system to continue working even some part failed
    - To scale out number of machine that can serve read request.

### Leader based replication (Master-slave replication, active/passive replication)
* One replica is the leader(master/primary) When client want to write, it send request to leader. Leader make local change, then send data change to all followers as a part of replication log or change system. Each follower update local copy of data.
* Client can request read from anyone.
![](images/leader_replica.PNG)

* PostgresSQL, MySQL, MongoDB, RethinkDB, Kafka all use this approach.

### Synchronous vs Asynchronous Replication
![](images/sync_async.PNG)
* Follower 1 is sync, 2 is async

* In synchronous we have guarantee that all copies are upto date with leader. If follower do not respond, write can not be completed. Generally 1 follower is synchronous and others asynchronous.
* Asynchronous replication is not guranteed and durable. But write is faster.

### Setting up new follower
* Take consistent snapshot of database at some time.
* Copy snapshot to new follower node.
* Follower connects to leader and ask for all data change that have happened since snapshot taken. It associate snapshot with leader's replication log
* Follower process backlog of data changes since backlog.